# WB4.3 Introducción a SQL

En esta cuaderno de trabajo se explorarán la base de datos `music.db` mediante consultas de SQL. La base de datos contiene tres tablas:
* `songs`, que contiene un identificador `_id`, el título de la canción `title` y el album `album`. 
* `albums`, que contiene un identificador `_id`, el nombre del album `name` y el artista `artist`. 
* `artists`, que contiene un identificador `_id` y su nombre `name`. 

Completa las siguientes instrucciones de acuerdo a lo visto en el tema.

## 1. Acceso a los Datos con Queries.

1. Crea la conexión y un cursor a la base de datos.

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("music.db")
cursor = conn.cursor()


2. Explora todas las columnas en todas antes mencionadas tablas. Muestra los primeros 10 registros.

In [ ]:
def mostrar_tabla(nombre_tabla):
    print(f"\nTabla: {nombre_tabla}")
    df = pd.read_sql_query(f"SELECT * FROM {nombre_tabla} LIMIT 10;", conn)
    display(df)

for tabla in ["songs", "albums", "artists"]:
    mostrar_tabla(tabla)


3. Iron Maiden tiene como `_id = 8`en la tabla de artistas. Emplea este id para identificar sus álbumes en la tabla `albums`. Observa que la columna `artist` hace referencia a los id de los artistas. 

In [ ]:
iron_maiden_albums = pd.read_sql_query("""
    SELECT * 
    FROM albums
    WHERE artist = 8;
""", conn)

display(iron_maiden_albums)

4. Muestras las canciones del album "The Number of the Beast". Presenta tus resultados en orden alfabético.

In [ ]:
songs_number_beast = pd.read_sql_query("""
    SELECT s.title
    FROM songs s
    JOIN albums a ON s.album = a._id
    WHERE a.name = 'The Number of the Beast'
    ORDER BY s.title ASC;
""", conn)

display(songs_number_beast)


## 2. Agregaciones y Agrupamientos de Datos.

1. ¿Cuantas canciones, albumes y artistas se tienen registrados en la base de datos?

In [ ]:
conteos = {}
for tabla in ["songs", "albums", "artists"]:
    cursor.execute(f"SELECT COUNT(*) FROM {tabla}")
    conteos[tabla] = cursor.fetchone()[0]

conteos


2. Agrupa la tabla albums por artista para identificar qué cuales son los 5 artistas que tienen más albumes.

In [ ]:
# Consulta SQL que utiliza una subconsulta para primero agregar los datos.
query_top_artists = """
    SELECT
        artists_table.name,
        album_counts.total_albums
    FROM
        artists AS artists_table
    INNER JOIN
        (
            SELECT
                artist AS artist_id,
                COUNT(_id) AS total_albums
            FROM
                albums
            GROUP BY
                artist_id
        ) AS album_counts ON artists_table._id = album_counts.artist_id
    ORDER BY
        album_counts.total_albums DESC
    LIMIT 5;
"""

# Ejecutar la consulta y cargarla en un DataFrame de pandas.
top_artists_df = pd.read_sql_query(query_top_artists, conn)

# Mostrar los 5 artistas con más álbumes.
display(top_artists_df)

3. Presenta el nombre de los artistas identificados en el punto anterior.

In [ ]:
top5_nombres = pd.read_sql_query("""
    SELECT ar.name, COUNT(al._id) AS num_albums
    FROM albums al
    JOIN artists ar ON al.artist = ar._id
    GROUP BY ar._id
    ORDER BY num_albums DESC
    LIMIT 5;
""", conn)

display(top5_nombres)

4. ¿Cuál es el album con más canciones? Investiga sobre este album en la red.

In [ ]:
# Definir la consulta utilizando una Expresión Común de Tabla (CTE).
# La CTE 'conteo_canciones_por_album' primero calcula cuántas canciones tiene cada álbum.
query_album_top = """
    WITH conteo_canciones_por_album AS (
        SELECT
            album AS album_id,
            COUNT(_id) AS numero_de_canciones
        FROM
            songs
        GROUP BY
            album
    )
    SELECT
        a.name AS nombre_album,
        c.numero_de_canciones
    FROM
        conteo_canciones_por_album c
    INNER JOIN
        albums a ON c.album_id = a._id
    ORDER BY
        c.numero_de_canciones DESC
    LIMIT 1;
"""

# Ejecutar la consulta SQL y cargar el resultado en un DataFrame.
df_top_album = pd.read_sql_query(query_album_top, conn)

# Mostrar el DataFrame con el resultado.
display(df_top_album)

## 3. Combinación de Tablas.

1. Muestra todas el nombre canciones del album "Seventh Son Of A Seventh Son" que tiene por '_id = 420'. Presenta la lista en el *track order* (`track`).

In [ ]:
# Definir la consulta SQL con un marcador de posición (?) para el parámetro.
# Esto previene la inyección de SQL y hace el código más reutilizable.
sql_query = """
    SELECT
        title,
        track
    FROM
        songs
    WHERE
        album = ?
    ORDER BY
        track ASC;
"""

# Asignar el ID del álbum a una variable para mayor claridad.
album_id_to_find = 420

# Ejecutar la consulta usando pd.read_sql y pasando el parámetro de forma segura.
# El argumento 'params' asocia la variable 'album_id_to_find' con el marcador '?'.
tracks_df = pd.read_sql(
    sql_query,
    conn,
    params=(album_id_to_find,)
)

# Mostrar el DataFrame resultante.
display(tracks_df)


2. Obtén todas las canciones de Aeroesmith en orden alfabético. Solo incluye el título en la salida ¿Cuántas son?

In [ ]:
# Definir el nombre del artista que queremos buscar.
artist_to_find = 'Aerosmith'

# La consulta utiliza una subconsulta para encontrar el ID del artista.
# Esto evita un JOIN adicional en la consulta principal.
# El nombre del artista se pasa como un parámetro (?) para seguridad.
query_songs_by_artist = """
    SELECT
        s.title
    FROM
        songs AS s
    INNER JOIN
        albums AS a ON s.album = a._id
    WHERE
        a.artist = (SELECT _id FROM artists WHERE name = ?)
    ORDER BY
        s.title ASC;
"""

# Ejecutar la consulta pasando el nombre del artista como parámetro.
songs_df = pd.read_sql(
    query_songs_by_artist,
    conn,
    params=(artist_to_find,)
)

# Mostrar los títulos de las canciones.
display(songs_df)

# Imprimir el conteo total de canciones encontradas.
print(f"Total de canciones de {artist_to_find}: {len(songs_df)}")

## Referencias
* Actividad basada en SQLite-Exercises: https://github.com/IancuIulian/SQLite-Exercises